## Parallel processing in the shell

Shell programming helps massively speed up data management tasks, and even more so with simple use of the [GNU `parallel`](https://www.gnu.org/software/bash/manual/html_node/GNU-Parallel.html) utility to execute bash commands in parallel. In its simplest form, this can be used to speed up common file operations, such as renaming, compression, decompression, and file transfer. Let's look at a common example -- calculating checksums to verify file integrity for data files. Calculating a hash checksum using the `shasum` command can be time consuming, especially when you have a lot of large files to work on. But it is a classic processor-limited task, and one that can be massively faster using `parallel`. 

First open a Bash Terminal, and then use "Split terminal" to create another, side-by-side. In the left terminal, run `htop`.

Now maximize your terminal, and in the right terminal run some `parallel` commands.

```bash
$ cd /home/shares/example-pdg-data
$ for fn in `ls *.gpkg`; do shasum -a 256 ${fn}; done

real	35.081s
user	32.745s
system	2.336s

$ ls *.gpkg | parallel "shasum -a 256 {}"

real    2.97s 
user    37.16s 
system  2.70s
```

The first invocation takes *35 seconds* to execute the tasks one at a time serially, while the second version only takes :tada: *3 seconds* :tada: to do the same tasks. Note that the computational time spent in `user` and `system` processing is about the same, with the major difference being that the user-space tasks were conducted on multiple cores in parallel, resulting in more than 10x faster performance, Using `htop`, you can see processor cores spiking in usage when the command is run.

## Task parallelization in Python

Python also provides a number of easy to use packages for concurrent processing. We will review two of these, `concurrent.futures` and `parsl`, to show just how easy it can be to parallelize your programs. [`concurrent.futures`](https://docs.python.org/3/library/concurrent.futures.html) is built right into the python3 release, and is a great starting point for learning concurrency.

FIrst, we'll get set up by creating a small function for timing other functions, for use as a [decorator](https://realpython.com/primer-on-python-decorators/).

In [ ]:
import time
from functools import wraps

def timethis(func):
    """ 
    Print the execution time for a function call
    """
    @wraps(func)
    def wrapped_method(*args, **kwargs):
        time_start = time.time()
        output = func(*args, **kwargs)
        time_end = time.time()
        print(f"{func.__name__}: {(time_end-time_start)*1000} ms")

        return output

    return wrapped_method

We're going to start with a task that is a little expensive to compute, and define it in a function. All this `task(x)` function does is to use numpy to create a fairly large range of numbers, and then sum them. 

In [ ]:
# Create a task function that we want to parallelize
def task(x):
    # Function code

We can start by executing this task function serially ten times with varying inputs. In this case, we create a function `run_serial` that takes a list of inputs to be run, and it calls the `task` function for each of those inputs. The `@timethis` decorator is a simple way to wrap the function with timing code so that we can see how long it takes to execute.

In [ ]:
# Create a function to run the task serially and time it
import numpy as np

def run_serial(task_list):
    # Function code


In this case, it takes around *25 seconds* to execute 10 tasks, depending on what else is happening on the machine and network.

So, can we make this faster using a multi-threaded parallel process? Let's try with `concurrent.futures`. The main concept in this package is one of a `future`, which is a structure which represents the value that will be created in a computation in the future when the function completes execution. With `concurrent.futures`, tasks are scheduled and do not block while they await their turn to be executed. Instead, threads are created and executed *asynchronously*, meaning that the function returns it's `future` potentially before the thread has actually been executed. Using this approach, the user schedules a series of tasks to be executed asynchronously, and keeps track of the futures for each task. When the future indicates that the execution has been completed, we can then retrieve the result of the computation.

In practice this is a simple change from our serial implementation. We will use the `ThreadPoolExecutor` to create a pool of workers that are available to process tasks. Each worker is set up in its own thread, so it can execute in parallel with other workers. After setting up the pool of workers, we use concurrent.futures `map()` to schedule each task from our `task_list` (in this case, an input value from 1 to 10) to run on one of the workers. As for all map() implementations, we are asking for each value in `task_list` to be executed in the `task` function we defined above, but in this case it will be executed using one of the workers from the `executor` that we created.

In [ ]:
# Now create a function to run it with multi-threaded execution
from concurrent.futures import ThreadPoolExecutor

@timethis
def run_threaded(task_list):
    # Function code

This execution took about *4 seconds*, which is about 6.25x faster than serial. Congratulations, you wrote your a multi-threaded python program!

In [ ]:
from concurrent.futures import ProcessPoolExecutor

@timethis
def run_processes(task_list):
    # Function code

## Exercise: Parallel downloads

In this exercise, we're going to parallelize a simple task that is often very time consuming -- downloading data. And we'll compare performance of simple downloads using first a serial loop, and then using two parallel execution libraries: `concurrent.futures` and `parsl`. We're going to see an example here where parallel execution won't always speed up this task, as this is likely an I/O bound task if you're downloading a lot of data. But we still should be able to speed things up a lot until we hit the limits of our disk arrays.

The data we are downloading is a pan-Arctic time series of TIF images containing rasterized Arctic surface water indices from:

> Elizabeth Webb. 2022. Pan-Arctic surface water (yearly and trend over time) 2000-2022. Arctic Data Center [doi:10.18739/A2NK3665N](https://doi.org/doi:10.18739/A2NK3665N). 

First, let's download the data serially to set a benchmark. The data files are listed in a table with their filename and identifier, and can be downloaded directly from the Arctic Data Center using their identifier. To make things easier, we've already provided a data frame with the names and identifiers of each file that could be downloaded.

In [ ]:
# Run this cell to get the list of files to be downloaded

import pandas as pd
import io

TESTDATA="""filename,identifier
"SWI_2007.tif","urn:uuid:5ee72c9c-789d-4a1c-95d8-cb2b24a20662"
"SWI_2019.tif","urn:uuid:9cd1cdc3-0792-4e61-afff-c11f86d3a9be"
"SWI_2021.tif","urn:uuid:14e1e509-77c0-4646-9cc3-d05f8d84977c"
"SWI_2020.tif","urn:uuid:1ba473ff-8f03-470b-90d1-7be667995ea1"
"SWI_2001.tif","urn:uuid:85150557-05fd-4f52-8bbd-ec5a2c27e23d"
"SWI_trend_2000_2021.tif","urn:uuid:ea99e31b-572a-4175-92d2-0a4a9cdd8366"
"SWI_2011.tif","urn:uuid:c45e3d93-e908-4f44-85c6-82c4c002a259"
"SWI_trend_2000_2022.tif","urn:uuid:a0aa394e-b0df-4daf-9011-09bba3c3d0b5"
"SWI_2015.tif","urn:uuid:5634766f-af81-46f8-acc5-e15f9cd2e3fb"
"SWI_2017.tif","urn:uuid:94a3fae2-f2f2-4e28-b51b-4a66b7b42331"
"SWI_2000.tif","urn:uuid:c7aac289-e2be-4be7-85df-6d2fdb5293d5"
"SWI_2018.tif","urn:uuid:07b9ca95-1463-414a-a3e0-7f466772d891"
"SWI_2014.tif","urn:uuid:695b949c-339d-4f51-a593-4d332aa86748"
"SWI_2005.tif","urn:uuid:e54142ed-dbb3-4b27-9756-d2262c6c069a"
"SWI_2012.tif","urn:uuid:57dcf9f5-2f25-4d37-a793-821edb2dda19"
"SWI_2008.tif","urn:uuid:75f20002-f4bf-4721-b480-d115d4042e79"
"resource_map_doi:10.18739_A2NK3665N.rdf","resource_map_doi:10.18739/A2NK3665N"
"Pan_Arctic_surface_water_yearly_and_trend_over.xml","doi:10.18739/A2NK3665N"
"SWI_2002.tif","urn:uuid:fe73a6b1-86af-495f-9292-ad22e86a4aa0"
"SWI_2013.tif","urn:uuid:938856af-d78d-4580-ab63-bb2ce8484c77"
"SWI_2006.tif","urn:uuid:705a3af3-fe5b-42d3-851f-e6093c15ee5c"
"SWI_2022.tif","urn:uuid:6c1ea229-8bfa-4e0f-bf6b-cff8e03865fb"
"SWI_2003.tif","urn:uuid:801e617c-4464-431d-b5c2-a8c16b543e7b"
"SWI_2016.tif","urn:uuid:bdf983fc-9864-4e08-9d3a-768afe36bf5f"
"SWI_2009.tif","urn:uuid:e1d9eb59-6e2b-4c0f-b048-206badd03fd1"
"SWI_2010.tif","urn:uuid:5bf32df3-83bf-4985-b682-560936247516"
"SWI_2004.tif","urn:uuid:7c1a1730-f1dc-43a7-96c2-fe99d480c87e"
"""

file_table = pd.read_csv(io.StringIO(TESTDATA), sep=",")
file_table.head()

### Serial

When you have a list of repetitive tasks, you may be able to speed it up by adding more computing power.  If each task is completely independent of the others, then it is pleasingly parallel and a prime candidate for executing those tasks in parallel, each on its own core.  For example, let's build a simple loop that downloads the data files that we need for an analysis. First, we start with the serial implementation.

In [ ]:
import urllib

# Now write a function using `urllib.urlretrieve` to download each file
def download_file(row):
    # Function code

@timethis
def download_serial(table):
    # Function code

# And execute it

In this code, we have one function (`download_file`) that downloads a single data file and saves it to disk. It is called iteratively from the function `download_serial`. The serial execution takes about *20-25 seconds*, but can vary considerably based on network traffic and other factors.

The issue with this loop is that we execute each download task sequentially, which means that only one of our processors on this machine is in use.  In order to exploit parallelism, we need to be able to dispatch our tasks and allow each to run at the same time, with one task going to each core.  To do that, we can use one of the many parallelization libraries in python to help us out.

### Multi-threaded with `concurrent.futures`

In this case, we'll use the same `download_file` function from before, but let's switch up and create a `download_threaded()` function to use `concurrent.futures` with a `ThreadPoolExecutor` just as we did earlier.

In [ ]:
# And now multi-threaded
from concurrent.futures import ThreadPoolExecutor

@timethis
def download_threaded(table):
    # Function code

# And run it to get 5 files, and print the results


Note how the "Downloading..." messages were printed immediately, but then it still took over 20 seconds to download the 5 files. This could be for several reasons, including that one of the files alone took that long (e.g., due to network congestion), or that there was a bottleneck in writing the files to disk (i.e., we could have been disk I/O limited). Or maybe the multithreaded executor pool didn't do a good job parallelizing the tasks. The trick is figuring out why you did or didn't get a speedup when parallelizing. So, let's try this another way, using a **multi-processing** approach, rather than multi-threading.

### Multi-process with `concurrent.futures`

You'll remember from earlier that you can execute tasks concurrently by creating multiple threads within one process (multi-threaded), or by creating and executing muliple processes. The latter creates more independence, as each of the executing tasks has their own memory and process space, but it also takes longer to set up. With `concurrent.futures`, we can switch to a multi-process pool by using a `ProcessPoolExecutor`, analogously to how we used `ThreadPoolExecutor` previously. So, some simple changes, and we're running multiple processes.

In [ ]:
# And a multi-process implementation
from concurrent.futures import ProcessPoolExecutor

@timethis
def download_process(table):
    # Function code

# And run it to get 3 files


Again, the output messages print almost immediately, but then later the processes finish and report that it took between *10 to 15 seconds* to run. Your mileage may vary. When I increase the number of files being downloaded to 10 or even to 20, I notice it is actually about the same, around *10-15 seconds*.  So, part of our time now is the overhead of setting up multiple processes. But once we have that infrastructure in place, we can make effective euse of the pool of processes to handle many downloads.

## Parallel processing with `parsl`

`concurrent.futures` is great and powerful, but it has its limits. Particularly as you try to scale up into the thousands of concurrent tasks, other libraries like [Parsl](https://parsl-project.org/) ([docs](https://parsl.readthedocs.io/)), [Dask](https://www.dask.org/), [Ray](https://www.ray.io/), and others come into play. They all have their strengths, but Parsl makes it particularly easy to build parallel workflows out of existing python code through it's use of decorators on existing python functions.

In addition, Parsl supports a lot of different kinds of [providers](https://parsl.readthedocs.io/en/stable/userguide/execution.html#execution-providers), allowing the same python code to be easily run multi-threaded using a `ThreadPoolExecutor` and via multi-processing on many different cluster computing platforms using the `HighThroughputExecutor`. For example, Parsl includes providers supporting local execution, and on Slurm, Condor, Kubernetes, AWS, and other platforms. And Parsl handles data staging as well across these varied environments, making sure the data is in the right place when it's needed for computations.

Similarly to before, we start by configuring an executor in parsl, and loading it. We'll use multiprocessing by configuring the `HighThroughputExecutor` to use our local resources as a cluster, and we'll activate our virtual environment to be sure we're executing in a consistent environment.

In [ ]:
# Set up the parsl executor
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

# Configure the parsl executor
activate_env = 'workon scomp'
htex_local = Config(
    executors=[
        HighThroughputExecutor(
            max_workers_per_node=15,
            provider=LocalProvider(
                worker_init=activate_env
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

We now have a live parsl executor (`htex_local`) that is waiting to execute processes. We tell it to execute processes by annotating functions with decorators that indicate which tasks should be parallelized. Parsl then handles the scheduling and execution of those tasks based on the dependencies between them. In the simplest case, we'll decorate our previous function for downloading a file with the `@python_app` decorator, which tells parsl that any function calls with this function should be run on the default executor (in this case, `htex_local`).

In [ ]:
# Copy the download function from above, and decorate it as a parsl @python_app


Now we just write regular python code that calls that function, and parsl handles the scheduling. Parsl app executors return an [`AppFuture`](https://parsl.readthedocs.io/en/stable/userguide/futures.html#appfutures), and we can call the `AppFuture.done()` function to determine when the future result is ready without blocking. Or, we can just block on `AppFuture.result()` which waits for each of the executions to complete and then returns the result.

In [ ]:
# Two functions, one to download our future object, and one to wait for them to return


When we're done, be sure to clean up and shutdown the `htex_local` executor, or it will continue to persist in your environment and utilize resources. Generally, an executor should be created when setting up your environment, and then it can be used repeatedly for many different tasks.

In [ ]:
# Shutdown the executor and clear the parsl engine


## Summary

In this lesson, we showed examples of computing tasks that are likely limited by the number of CPU cores that can be applied, and we reviewed the architecture of computers to understand the relationship between CPU processors and cores.  Next, we reviewed the way in which traditional sequential loops can be rewritten as functions that are applied to a list of inputs both serially and in parallel to utilize multiple cores to speed up computations. We reviewed the challenges of optimizing code, where one must constantly examine the bottlenecks that arise as we improve cpu-bound, I/O bound,and memory bound computations.

## Are we done yet?

A foray into blocking and non-blocking operations, using `future.done()`.

In [ ]:
# Required packages and parsl configuration
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

# Configure the parsl executor
activate_env = 'workon scomp'
htex_local = Config(
    executors=[
        HighThroughputExecutor(
            max_workers_per_node=5,
            provider=LocalProvider(
                worker_init=activate_env
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

In [ ]:
# define a task function

@python_app
def do_stuff(x):
    # implementation here

In [ ]:
# Launch tasks with parsl, and check if each is done (once)


In [ ]:
# Shutdown the executor and clear the parsl engine
htex_local.executors[0].shutdown()
parsl.clear()